# 13-json输出解析器

此输出解析器允许用户指定任意 JSON 架构并查询 LLMs 以获得符合该架构的输出。

请记住，大型语言模型是有漏洞的抽象！您必须使用具有足够容量的 LLM 来生成格式正确的 JSON。在OpenAI家族中，达芬奇可以可靠地完成任务，但居里的能力已经急剧下降。

您可以选择使用 Pydantic 来声明您的数据模型。

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List, Optional
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("LOCAL_API_KEY")
base_url = os.getenv("LOCAL_API_BASE")

llm = ChatOpenAI(api_key=api_key, base_url=base_url, temperature=0.3, max_tokens=8192)


/home/libing/miniconda3/envs/kk_langchain_wsl/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
# 自定义输出的数据结构
class Book(BaseModel):
    title: str = Field(description="书名")
    author: str = Field(description="作者")
    year: int = Field(description="出版年份")
    price: float = Field(description="价格")
    description: str = Field(description="书的简介")

# 创建输出解析器
output_parser = JsonOutputParser(pydantic_object=Book)

format_instructions = output_parser.get_format_instructions()
format_instructions = """输出符合以下格式的JSON字符串实例: \n
```JSON
{
    "title": "书名",
    "author": "作者",
    "year": "出版年份",
    "price": "价格",
    "description": "书的简介"
}
```
"""

# 定义提示词模板
prompt = PromptTemplate(
    template="{format_instructions}\n请列出{query}的信息",
    input_variables=["query"],
    partial_variables={"format_instructions": format_instructions}
)

# 创建链
chain = prompt | llm | output_parser

In [3]:
chain.invoke({"query": "如何开始学习中国历史"})

{'title': '中国历史入门指南',
 'author': '李明',
 'year': '2023年',
 'price': '￥98.00',
 'description': '本书旨在为初学者提供一套全面的中国历史学习框架。从远古的传说时代到现代中国的发展，本书涵盖了各个时期的重要事件、人物和文化成就。通过丰富的图表和插图，以及易于理解的叙述方式，读者可以轻松地了解中国的悠久历史。此外，书中还提供了进一步阅读的建议和在线资源链接，帮助读者深入探索感兴趣的历史话题。'}

In [4]:
chain.invoke({"query": "红楼梦"})

{'title': '红楼梦',
 'author': '曹雪芹',
 'year': '1784年（乾隆年间）',
 'price': '价格信息不详',
 'description': '《红楼梦》是中国古典文学四大名著之一，是一部以贾、王、史、薛四大家族的兴衰为背景的长篇小说。它通过描写贾宝玉和林黛玉的爱情悲剧，反映了封建社会的腐朽与没落，以及贵族家庭的衰败过程。该书在艺术上达到了很高的成就，被誉为中国古代小说的巅峰之作。'}